In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
df = pd.read_csv('bom.movie_gross.csv.gz')
movies = pd.read_csv('imdb.title.basics.csv.gz')

# this is to fill NA values so I can add gross columns together for each movie
values = {'studio': 'NA', 'domestic_gross': 0, 'foreign_gross': 0}
df = df.fillna(value=values)

# reassign billion dollar movies to values in billions
df.at[1872, 'foreign_gross'] = 1131600000
df.at[1873, 'foreign_gross'] = 1019400000
df.at[1874, 'foreign_gross'] = 1163000000
df.at[2760, 'foreign_gross'] = 1010000000
df.at[3079, 'foreign_gross'] = 1369500000

# make all foreign_gross values an integer
df['foreign_gross'] = df['foreign_gross'].astype(int)
df['total_gross'] = df['foreign_gross'] + df['domestic_gross']

#drop all columns but title and total_gross and retrieve the first 100 most grossed movies
new_df = df.drop(['studio', 'domestic_gross', 'foreign_gross', 'year'], axis = 1)

# make all total_gross numbers display without 'e'
pd.options.display.float_format = "{:.2f}".format

#organize from most profittable to least
most_profittable = new_df.nlargest(4000, columns = 'total_gross')


movies = pd.read_csv('imdb.title.basics.csv.gz')
new_movies = movies
new_movies = new_movies.drop(columns = ['original_title', 'runtime_minutes', 'start_year', 'tconst'])
new_movies.rename(columns = {'primary_title':'title'}, inplace = True)

# combined tables with 'title', 'total_gross', 'genre_ids', 'popularity'
two_tables = pd.merge(most_profittable, new_movies, on = 'title', how = 'inner')
# drop unnecessary columns
two_tables

# make a dataframe for 'popularity'
popularity = pd.read_csv('tmdb.movies.csv.gz')
popularity = popularity.drop(columns = ['Unnamed: 0','genre_ids','id','original_language','original_title','release_date','vote_average','vote_count'])
popularity

# combine 'two_tables' with 'popularity'
combined_data = pd.merge(two_tables, popularity, on = 'title', how = 'inner')
combined_data

# take out duplicate title, used the last title because it had the highest popularity, the total_gross was the same
combined_data['title'] = combined_data.title.drop_duplicates(keep = 'last')
combined_data = combined_data.dropna(subset = ['title'], axis = 0, how = 'any')
combined_data

# Locate any na values in data
combined_data.isna().any()
combined_data.loc[combined_data['genres'].isnull()]

# alter data in column to appropriate values
combined_data.at[304, 'genres'] = 'Action, Sci-Fi'
combined_data.at[3240, 'genres'] = 'Comedy'
combined_data.at[3093, 'genres'] = 'Drama'
combined_data.at[2618, 'genres'] = 'Drama, Romance'
combined_data.at[2388, 'genres'] = 'Crime, Drama'
combined_data.at[2347, 'genres'] = 'Drama, Thriller'
combined_data.at[2182, 'genres'] = 'Drama, Thriller'
combined_data.at[2049, 'genres'] = 'Drama'
combined_data.at[1797, 'genres'] = 'Animation, Family'
combined_data.at[1411, 'genres'] = 'Drama, Horror'
combined_data.at[1280, 'genres'] = 'Comedy, Romance'
combined_data.at[752, 'genres'] = 'Action, Sci-Fi'
combined_data.at[661, 'genres'] = 'Action, Thriller'
combined_data.at[643, 'genres'] = 'Comedy, Romance'
combined_data.at[553, 'genres'] = 'Thriller, Drama'
combined_data.at[62, 'genres'] = 'Animation, Comedy'

# seperate all genres
u = str(set(list(combined_data['genres'])))
u.count('Adventure')
combined_data["genres"]= combined_data["genres"].str.split(",", n = 1, expand = True)

# change total_gross values to values in millions and rename it
combined_data['total_gross'] = combined_data['total_gross'].map(lambda x: int(x))
combined_data['total_gross'] = combined_data['total_gross'].map(lambda x: x / 10000)
combined_data=combined_data.rename(columns = {'total_gross': "total_gross_in_ten_thousands's"})


#graph 1

plt.figure(figsize = (10,10))
sns.boxplot(x = 'popularity', y ='genres', data = combined_data.nlargest(4000, 'popularity'), color = 'b', showfliers = False)
plt.title("Popularity for Each Genre")
plt.xlabel("Popularity Scores")
plt.ylabel("Genres")

# graph 2
plt.figure(figsize = (10,10))
sns.boxplot(x = "total_gross_in_ten_thousands's", y = 'genres', data = combined_data.nlargest(4000, "total_gross_in_ten_thousands's"), color = 'b', showfliers=False)
plt.title("How Much Money Each Genre of Movie Earns")
plt.xlabel("Amount in Tens of Thousands")
plt.ylabel("Genres")
plt.legend(['each blip is a movie'])





# combined_data




,title,total_gross,genres
0,Avengers: Infinity War,2048300000.00,"Action,Adventure,Sci-Fi"
1,Jurassic World,1671700000.00,"Action,Adventure,Sci-Fi"
2,Furious 7,1516000000.00,"Action,Crime,Thriller"
3,Avengers: Age of Ultron,1405400000.00,"Action,Adventure,Sci-Fi"
4,Black Panther,1347000000.00,"Action,Adventure,Sci-Fi"
...,...,...,...
3361,Death of a Superhero,600.00,"Animation,Drama"
3362,2:22,400.00,Thriller
3363,Satanic,300.00,Horror
3364,The Chambermaid,300.00,Drama
